In [ ]:
!pip uninstall -y keras
# installing GPU enabled TensorFlow 2.15.1
!pip install --upgrade "tensorflow[and-cuda]==2.15.1"
#reinstalling Larq but ignoring its old <2.14 version
!pip install --no-deps larq==0.13.3

Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.8/845.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install --force-reinstall --no-deps \
     numpy==1.25.2 \
     ml-dtypes==0.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2


In [ ]:
!pip install --quiet terminaltables==3.1.10

In [ ]:
import tensorflow as tf, larq as lq
print("TensorFlow :", tf.__version__)
print("Larq       :", lq.__version__)


TensorFlow : 2.15.1
Larq       : 0.13.3


In [ ]:
from google.colab import files
uploaded = files.upload()  # .h5 model

Saving mnist_model.h5 to mnist_model.h5


In [ ]:
import time, numpy as np
import tensorflow as tf, larq as lq
from tensorflow.keras.datasets import mnist

# Configurable batch size
batch_size = 1

# model loading
custom_objects = {
    "QuantDense": lq.layers.QuantDense,
    "ste_sign": lq.quantizers.SteSign,
    "weight_clip": lq.constraints.WeightClip,
}

model = tf.keras.models.load_model("mnist_model.h5", custom_objects=custom_objects)

# preparing input batch
(_, _), (test_images, _) = mnist.load_data()
x = test_images[:batch_size].reshape(batch_size, 28 * 28).astype("float32")
x = x / 127.5 - 1  # Normalize to [-1, 1]
x_tensor = tf.convert_to_tensor(x)

# Graph-compiled inference
@tf.function
def infer(x):
    return model(x, training=False)

_ = infer(x_tensor)  # warm-up

# timed loop
num_runs = 100
timings = []
for _ in range(num_runs):
    t0 = time.perf_counter_ns()
    _ = infer(x_tensor)
    timings.append(time.perf_counter_ns() - t0)

# Function for printing stats
def print_stats(label, timings_ns, batch):
    arr = np.asarray(timings_ns)
    mean_total = arr.mean()
    std_total = arr.std()
    min_total = arr.min()
    max_total = arr.max()

    mean_per_img = mean_total / batch
    print(f"\n{label} Benchmark over {len(arr)} runs (batch size = {batch})")
    print("-----------------------------------------------------------------------")
    print(f"Mean total: {mean_total:.0f} ns ({mean_total / 1e6:.6f} ms)")
    print(f"Per image : {mean_per_img:.0f} ns ({mean_per_img / 1e6:.6f} ms)")
    print(f"Std Dev   : {std_total:.0f} ns")
    print(f"Min       : {min_total:.0f} ns")
    print(f"Max       : {max_total:.0f} ns")

#results
print_stats("Optimized CPU", timings, batch_size)



Optimized CPU Benchmark over 100 runs (batch size = 1)
-----------------------------------------------------------------------
Mean total: 1604866 ns (1.604866 ms)
Per image : 1604866 ns (1.604866 ms)
Std Dev   : 288572 ns
Min       : 1077753 ns
Max       : 2510309 ns


In [ ]:
#for printing CPU info
from psutil import *
print("Number of CPU: ", cpu_count())
!cat /proc/cpuinfo

Number of CPU:  2
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache